In [9]:

geo_df = gpd.read_file(r'C:\Users\ttk8t\python\kokudo\data\mesh\1km_mesh_2024_01_SHP', encoding="utf-8")

In [1]:
import geopandas as gpd
import pandas as pd
geo_dfs = gpd.GeoDataFrame()

for i in range(47):
    geo_df = gpd.read_file(fr'C:\Users\ttk8t\python\kokudo\data\mesh\1km_mesh_2024_{i+1:02d}_SHP', encoding="utf-8")
    geo_dfs = pd.concat([geo_dfs,geo_df],axis=0)

In [17]:
import pandas as pd
# df_train_2022 = pd.read_csv('../data/2022/train_2022.csv')
# df_train_2021 = pd.read_csv('../data/2021/train_2021.csv')
# df_train_2020 = pd.read_csv('../data/2020/train_2020.csv')
# df_train_2019 = pd.read_csv('../data/2019/train_2019.csv')
test_df = pd.read_csv('../data/2023/test.csv')
train_df = pd.read_csv('../data/train.csv')
# train_df = pd.concat([df_train_2022,df_train_2021,df_train_2020,df_train_2019],axis=0)

C:\Users\ttk8t\AppData\Local\Temp\ipykernel_2272\2929164813.py:6: DtypeWarning: Columns (47,56,57,64,147) have mixed types. Specify dtype option on import or set low_memory=False.
  test_df = pd.read_csv('../data/2023/test.csv')
C:\Users\ttk8t\AppData\Local\Temp\ipykernel_2272\2929164813.py:7: DtypeWarning: Columns (63) have mixed types. Specify dtype option on import or set low_memory=False.
  train_df = pd.read_csv('../data/train.csv')


In [26]:
import pandas as pd
import geopandas as gpd
from shapely import wkt
from shapely.geometry import Point
geometry_tune = [Point(xy) for xy in zip(test_df['lon'], test_df['lat'])]
tune_gdf = gpd.GeoDataFrame(test_df, geometry=geometry_tune, crs="EPSG:4326")

In [27]:
# 2. geo.csv を GeoDataFrame に変換（面データを作成）
# WKT形式の文字列（POLYGON...）を幾何学オブジェクトに変換
# geo_df['geometry_obj'] = geo_df['geometry'].apply(wkt.loads)
# geo_gdf = gpd.GeoDataFrame(geo_df, geometry='geometry_obj', crs="EPSG:4326")

# 3. 空間結合を実行
# op='within' : 点が面の中に「含まれる」場合に結合
# how='left'  : tune.csvの全データを残し、該当するgeo情報のみ付与
merged_gdf = gpd.sjoin(tune_gdf, geo_dfs, how='left', predicate='within')

# 結果の確認
matched_count = merged_gdf['index_right'].notnull().sum()

C:\Users\ttk8t\AppData\Local\Temp\ipykernel_2272\1783884494.py:9: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: EPSG:4326
Right CRS: EPSG:6668

  merged_gdf = gpd.sjoin(tune_gdf, geo_dfs, how='left', predicate='within')


In [28]:
gdf = merged_gdf[~merged_gdf.index.duplicated(keep='first')]

In [29]:
len(merged_gdf)

114641

In [30]:
len(tune_gdf)

112437

In [31]:
len(gdf)

112437

In [35]:
gdf['unit_name'] = gdf['unit_name'].astype(str)

c:\Users\ttk8t\python\kokudo\.venv\Lib\site-packages\geopandas\geodataframe.py:1969: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [36]:
gdf.to_parquet('../data/test_mesh.parquet')